In [42]:
from bson.objectid import ObjectId
from pymongo import MongoClient
import numpy as np
import jaro

def test_filter(name, site1, raw_one1, site2, raw_one2):
    inst = jaro.jaro_winkler_metric(raw_one1.get('author_inst').split(";")[raw_one1.get('author').split(";").index(name)], raw_one2.get('author_inst').split(";")[raw_one2.get('author').split(";").index(name)])
    weight = 0
    
    if raw_one1.get('title') == raw_one2.get('title') or inst >= 0.8:
        weight = 7
    else:
        if site1 == 'NTIS' :
            #NTIS 
            print('ntis')
            if raw_one1.get('rsc') == None:
                coauthor1 = 0
            else:
                coauthor1 = raw_one1.get('rsc').split(";")

            if raw_one1.get('koKeyword') == None:
                keyword1 = ""
            else:
                keyword1 = raw_one1.get('koKeyword').split(",")

        else:
            journal1 = raw_one1.get('journal')
            issInst1 = raw_one1.get('issue_inst')
            issYear1 = int(raw_one1.get('issue_year')[:4])
            coauthor1 = raw_one1.get('author').split(";")[1:-1]
            keyword1 = raw_one1.get('paper_keyword').replace(" ", "").split(".")

        if site2 == 'NTIS' :
            #NTIS
            print('ntis')
            if raw_one2.get('rsc') == None:
                coauthor2 = 0
            else:
                coauthor2 = raw_one2.get('rsc').split(";")

            if raw_one2.get('koKeyword') == None:
                keyword2 = ""
            else:
                keyword2 = raw_one2.get('koKeyword').split(",")

        else:
            journal2 = raw_one2.get('journal')
            issInst2 = raw_one2.get('issue_inst')
            issYear2 = int(raw_one2.get('issue_year')[:4])
            coauthor2 = raw_one2.get('author').split(";")[1:-1]
            keyword2 = raw_one2.get('paper_keyword').replace(" ", "").split(".")

                
        if journal1 == journal2 and issInst1 == issInst2:
            weight += 1
        else:
            weight -= 1
        
        print(f'Site: {site1} | Journal: {journal1} | Inst: {issInst1} | Year: {issYear1} | Coauthor: {coauthor1} | keyword: {keyword1}')
        print(f'Site: {site2} | Journal: {journal2} | Inst: {issInst2} | Year: {issYear2} | Coauthor: {coauthor2} | keyword: {keyword2}')

        yop = -(2*(abs(issYear1-issYear2)/10)-1)
        co_authorship = 1 - np.exp(-len([i for i in coauthor1 if i in coauthor2]))
        keyword = 1 - np.exp(-len([i for i in keyword1 if i in keyword2]))

        print(f'yop: {yop} | co_authorship: {co_authorship} | keyword: {keyword} ')

        weight += yop + co_authorship + keyword

    return weight

In [43]:
client = MongoClient('mongodb://203.255.92.141:27017', authSource='admin')

name = '최도진'
site1 = 'Scienceon'
site2 = 'Scienceon'
input_paper1 = ObjectId('61939443e9b04a9d64abe055')
input_paper2 = ObjectId('61939447e9b04a9d64abe06a')

if site1 == 'Scienceon' :
    site_raw1 = client['SCIENCEON']['Rawdata']
elif site1 == 'NTIS' :
    site_raw1 = client['NTIS']['Rawdata']
elif site1 == 'DBPIA' :
    site_raw1 = client['DBPIA']['Rawdata']

if site2 == 'Scienceon' :
    site_raw2 = client['SCIENCEON']['Rawdata']
elif site2 == 'NTIS' :
    site_raw2 = client['NTIS']['Rawdata']
elif site2 == 'DBPIA' :
    site_raw2 = client['DBPIA']['Rawdata']

input_raw1 = site_raw1.find_one({ '_id' : input_paper1 })
input_raw2 = site_raw2.find_one({ '_id' : input_paper2 })

print(test_filter(name, site1, input_raw1, site2, input_raw2))


7
